<a href="https://colab.research.google.com/github/NarendraPolimera/MDSC-301-ML/blob/main/Scraping_Smart_watch_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
from bs4 import BeautifulSoup
import requests

# User Agent

In [ ]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

# Scraping

In [ ]:
def scrapePage(page):
    review_stars = []
    reviews = []
    nam=[]
    review_title = []
    #for each card in the page
    page = BeautifulSoup(page.text, "html.parser")
    for card in page.find_all("div",attrs={'data-hook':'review'},class_='a-section review aok-relative'):
        # find name of the customer
        name = card.find("span",class_='a-profile-name')
        if name ==None:
          continue
        # find review_title
        review_titl = card.find('a', class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold").findChild()
        if review_titl == None:
           continue
        #find star rating
        stari = card.find("i",attrs={'data-hook':'review-star-rating'})
        if stari==None:
          continue
        stars = stari.find("span",class_='a-icon-alt')
        #find review
        review = card.find("span",attrs={'data-hook':'review-body','class':'a-size-base review-text review-text-content'})
        # review = review.find("span")
        if review == None or stars==None:
          continue
        review_stars.append(int(stars.text[0]))
        reviews.append(review.text.strip())
        nam.append(name.text)
        review_title.append(review_titl.text)
        
    return reviews, review_stars, nam, review_title

In [ ]:
from tqdm.notebook import tqdm_notebook

name=[]
review_header=[]
reviews=[]
stars=[]
LIMIT=1500
count200 = 0
# Amazon Page link
url_link='https://www.amazon.in/boAt-Wave-Call-Dedicated-Multi-Sport/product-reviews/B0B5B6PQCT/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='
i=2
for i in tqdm_notebook(range(1,LIMIT),desc='Scraping:'):
  page = requests.get(url_link + str(i), headers=HEADERS)
  i+=1
  if page.status_code==200:
    count200+=1
    reviewsp, starsp, namp, review_title  = scrapePage(page)
    reviews.extend(reviewsp)
    stars.extend(starsp)
    name.extend(namp)
    review_header.extend(review_title)

print("# PAGE STATUS 200: ",count200,"/",LIMIT)




Scraping::   0%|          | 0/1499 [00:00<?, ?it/s]

# PAGE STATUS 200:  1499 / 1500


In [ ]:
reviews

["[Update: Sept 29] boAt seems to have heard the feedback 😀 and updated the watch faces. Now there are a few good ones to pick but a slight improvement over the ones the watch earlier had. Bit Impressed. Added one star for it.Not a very polished product. The fonts are tacky and look ugly in the watch for notification text and caller name text. The watch faces are a lazy design job. No point having 150 watch faces if only 10-20 are good or usable. Rest are useless.[UPDATE: Oct 11] boAt seems to be on a smartwatch launch spree. And thanks to that, they are constantly updating the phone app and the watch faces. Now you have many good looking watch faces to choose from. So overall, still the Smartwatch UI needs work. But atleast you have good watch faces.Good:- Screen quality- Battery life- Looks and the metallic frame- Call clarity- BT calling at the price- Reasonably accurate sensorsBad:- The UI, fonts - they are all very ugly and tacky. They could have tried to give the Chinese software

In [ ]:
len(review_header)

1330

In [ ]:
print(len(stars),len(reviews))

1330 1330


# Saving in .csv

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df['Names']= name
df['stars']=stars
df['Review_header']=review_header
df['reviews']=reviews

In [ ]:
df.head()

,Names,stars,Review_header,reviews
0,AV,4,Not Polished Enough. (Improving with updates),[Update: Sept 29] boAt seems to have heard the...
1,Durga mani kartheek,4,One of the best build quality of this price range,The heart rate sensor is 90% to 85% accurateI ...
2,Mihika a.,4,They didn't send the charger with the watch.,Boat smart watch is good but I didn't get the ...
3,Neeraj Maurya,4,This is nice but whenever connected through BT...,This is nice in that price cap. but I think sh...
4,aditya,4,Good in comparison with Fireboltt.,Used fireboltt watch but was not satisfied wit...


In [ ]:
df.stars.unique()

array([4, 3, 5, 2, 1])

In [ ]:
df.shape

(1330, 4)

In [ ]:
df = df.drop_duplicates()
df

,Names,stars,Review_header,reviews
0,AV,4,Not Polished Enough. (Improving with updates),[Update: Sept 29] boAt seems to have heard the...
1,Durga mani kartheek,4,One of the best build quality of this price range,The heart rate sensor is 90% to 85% accurateI ...
2,Mihika a.,4,They didn't send the charger with the watch.,Boat smart watch is good but I didn't get the ...
3,Neeraj Maurya,4,This is nice but whenever connected through BT...,This is nice in that price cap. but I think sh...
4,aditya,4,Good in comparison with Fireboltt.,Used fireboltt watch but was not satisfied wit...
...,...,...,...,...
1325,Prasanna M,5,Best,"Its a very nice product, easy to use. Good bat..."
1326,Aayu Tyagi,2,Good,The watch is verymuch similar to other Boat wa...
1327,BULBUL KASHMIRI,4,Best at this Range,Overall the product is good. I am satisfied w...
1328,Saravan Singh,5,Awesome Smartwatch,I have gifted this watch to my wife and she re...


In [ ]:
df['stars'].value_counts()

5    572
1    362
4    182
3    113
2     99
Name: stars, dtype: int64

In [ ]:
df.describe()

,stars
count,1328.000000
mean,3.378765
std,1.698104
min,1.000000
25%,1.000000
50%,4.000000
75%,5.000000
max,5.000000


In [ ]:
df.to_csv('./boAt Wave Call Smart Watch Smart Talk with Advanced Dedicated Bluetooth Calling Chip_raw.csv',index=False)